#### Example Ensembles: Australian Credit Card Assessment

In this example it will be compared 4 classifiers:
- Multilayer perceptron.
- Normal Ensemble.
- Ensemble trained with Negative Correlation (Xin Yao, 1999).
- Ensemble trained with Correntropy.

## Data

This data base concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.
  
- Number of Instances: 690
- Number of Attributes: 14 + class attribute
- Attribute Information:   THERE ARE 6 NUMERICAL AND 8 CATEGORICAL ATTRIBUTES.
- Exist 2 classes and its distribution is:
    * +: 307 (44.5%)    CLASS 2
    * -: 383 (55.5%)    CLASS 1
- there are 6 numerical and 8 categorical attributes. the labels have been changed for the convenience of the statistical algorithms. For example, attribute 4 originally had 3 labels p,g,gg and these have been changed to labels 1,2,3.

- Missing Attribute Values: 37 cases (5%) HAD one or more missing values. The missing values from particular attributes WERE:
    * A1:  12
    * A2:  12
    * A4:   6
    * A5:   6
    * A6:   9
    * A7:   9
    * A14: 13
 
  These were replaced by the mode of the attribute (categorical) and mean of the attribute (continuous)


In [1]:
import numpy as np
import theano

import os
import sys

sys.path.insert(0, os.path.abspath('../../..'))  # load deepensemble library

from deepensemble.models.sequential import Sequential
from deepensemble.layers.dense import Dense
from deepensemble.utils import *
from deepensemble.metrics import FactoryMetrics
from deepensemble.utils.utils_functions import ActivationFunctions

path_data = r'data/australian.dat'
data = np.genfromtxt(path_data, delimiter=' ')

classes_names = np.asarray(['clase 1', 'clase 2'], dtype='<U10')
data_input    = np.asarray(data[:, 0:-1], dtype=theano.config.floatX)
data_target   = classes_names[np.asarray(data[:, -1:], dtype=int)]

ImportError: libcblas.so: cannot open shared object file: No such file or directory

### Experimental Setup

The data set was partitioned into two sets: a training set and a testing set. The first 518 examples were used for the training set, and the remaining 172 examples for the testing set. The input attributes were rescaled to between 0.0 and 1.0 by a linear function. (**Ensemble learning via negative correlation, Y. Liua, X. Yao**)

In [ ]:
from sklearn import preprocessing

data_input_norm = preprocessing.MinMaxScaler().fit_transform(data_input)

input_train = data_input_norm[0:517]
input_test = data_input_norm[518:690]
target_train = data_target[0:517]
target_test = data_target[518:690]

## Training MLP

This model has ten neurons in hidden layer and the output layer two neurons with **one hot encoding**. The cost function is MSE and update function is Stochastic gradient descendent. 

In [ ]:
mlp = Sequential("mlp", "classifier", classes_names)
mlp.add_layer(Dense(n_input=data_input.shape[1], n_output=40, activation=ActivationFunctions.sigmoid))
mlp.add_layer(Dense(n_output=len(classes_names), activation=ActivationFunctions.sigmoid))
mlp.append_cost(mse, name='MSE')
mlp.set_update(sgd, name='SGD',  learning_rate=0.1)
mlp.compile(fast=False)

metrics_mlp = FactoryMetrics.get_metric(mlp)

max_epoch = 250
validation_jump = 5

Logger().reset()
for i in range(25):                      
    # training
    metric = mlp.fit(input_train, target_train, max_epoch=max_epoch, batch_size=32, early_stop=False)
    # Compute metrics
    metrics_mlp.append_prediction(input_test, target_test)
    metrics_mlp.append_metric(metric)
    
    # Reset parameters
    mlp.reset()

print("FINISHED MLP!")

## Results MLP

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt

plt.style.use('ggplot')

metrics_mlp.classification_report()
metrics_mlp.plot_confusion_matrix()
metrics_mlp.plot_cost(max_epoch, "Cost training MLP")
metrics_mlp.plot_scores(max_epoch, "Accuracy training MLP")

plt.tight_layout()

## Ensemble

In [ ]:
from deepensemble.models import EnsembleModel
from deepensemble.combiner import *

# Create Ensemble
ensemble = EnsembleModel(name="Ensemble")

# Create models for ensemble
for i in range(4):
    net = Sequential("net%d_ens" % i, "classifier", classes_names)
    net.add_layer(Dense(n_input=data_input.shape[1], n_output=10, activation=ActivationFunctions.sigmoid))
    net.add_layer(Dense(n_output=len(classes_names), activation=ActivationFunctions.sigmoid))
    net.append_cost(mse, name='MSE')
    net.set_update(sgd, name='SGD', learning_rate=0.1)
    ensemble.append_model(net)

ensemble.set_combiner(PluralityVotingCombiner())
ensemble.compile(fast=False)

metrics_ensemble = FactoryMetrics.get_metric(ensemble)

Logger().reset()
for i in range(25):                      
    # training
    metrics = ensemble.fit(input_train, target_train,
                            max_epoch=max_epoch, batch_size=32, early_stop=False)
    # Compute metrics
    metrics_ensemble.append_prediction(input_test, target_test)
    metrics_ensemble.append_prediction_per_model(input_test, target_test)
    metrics_ensemble.append_metric(metrics)
    
    # Reset parameters of all ensemble's models
    ensemble.reset()

print("FINISHED!")

### Result Ensemble

In [ ]:
metrics_ensemble.classification_report()
metrics_ensemble.diversity_report()
metrics_ensemble.plot_confusion_matrix()
metrics_ensemble.plot_cost(max_epoch, "Cost training Ensemble")
metrics_ensemble.plot_costs(max_epoch, "Cost training Ensemble per models")
metrics_ensemble.plot_scores(max_epoch, "Accuracy training Ensemble per models")

plt.tight_layout()

## Training Ensemble - Negative Correlation (Xin Yao, 1999)

The ensemble architecture used in the experiments has four networks. Each individual network is a feedforward network with one hidden layer. Both the hidden node function and the output node function are defined by the logistic function. All the individual networks have ten hidden nodes. The number of training epochs was set to 250. The strength parameter of negative correlations was set to 1.0. These parameters were chosen after limited preliminary experiments. They are not meant to be optimal.

In [ ]:
# Create Ensemble
ensemble_nc = EnsembleModel(name="Ensemble NC")

# Create models for ensemble
for i in range(4):
    net = Sequential("net%d_ens_nc" % i, "classifier", classes_names)
    net.add_layer(Dense(n_input=data_input.shape[1], n_output=10, activation=ActivationFunctions.sigmoid))
    net.add_layer(Dense(n_output=len(classes_names), activation=ActivationFunctions.sigmoid))
    net.append_cost(mse, name='MSE')
    net.set_update(sgd, name='SGD', learning_rate=0.1)
    ensemble_nc.append_model(net)

ensemble_nc.add_cost_ensemble(fun_cost=neg_corr, name='NCL', lamb=1.0)  # adds neg correlation in all models
ensemble_nc.set_combiner(PluralityVotingCombiner())
ensemble_nc.compile(fast=False)

metrics_ensemble_nc = FactoryMetrics.get_metric(ensemble_nc)

Logger().reset()
for i in range(25):                      
    # training
    metrics = ensemble_nc.fit(input_train, target_train,
                                max_epoch=max_epoch, batch_size=32, early_stop=False)
    # Compute metrics
    metrics_ensemble_nc.append_prediction(input_test, target_test)
    metrics_ensemble_nc.append_prediction_per_model(input_test, target_test)
    metrics_ensemble_nc.append_metric(metrics)
    
    # Reset parameters of all ensemble's models
    ensemble_nc.reset()

print("FINISHED!")

### Results Ensemble - Negative Correlation

In [ ]:
metrics_ensemble_nc.classification_report()
metrics_ensemble_nc.diversity_report()
metrics_ensemble_nc.plot_confusion_matrix()
metrics_ensemble_nc.plot_cost(max_epoch, "Cost training Ensemble-NC")
metrics_ensemble_nc.plot_costs(max_epoch, "Cost training Ensemble-NC per models")
metrics_ensemble_nc.plot_scores(max_epoch, "Accuracy training Ensemble-NC per models")

plt.tight_layout()

## Training Ensemble - Correntropy

In [ ]:
# Create Ensemble
ensemble_corrpy = EnsembleModel(name="Ensemble Correntropy")

# Create models for ensemble
for i in range(4):
    net = Sequential("net%d_ens_corrpy" % i, "classifier", classes_names)
    net.add_layer(Dense(n_input=data_input.shape[1], n_output=10, activation=ActivationFunctions.sigmoid))
    net.add_layer(Dense(n_output=len(classes_names), activation=ActivationFunctions.softmax))
    net.append_cost(kullback_leibler_generalized, name="KLG")
    net.set_update(sgd, name='SGD', learning_rate=0.001)
    
    net.compile()
    net.fit(input_train, target_train, max_epoch=max_epoch, batch_size=32, early_stop=False)
    
    ensemble_corrpy.append_model(net)
    

ensemble_corrpy.add_cost_ensemble(neg_correntropy, name="NEG_CORRPY", lamb=1.0)  # adds correntropy cost in all models
ensemble_corrpy.set_combiner(PluralityVotingCombiner())
ensemble_corrpy.compile(fast=False)

metrics_ensemble_corrpy = FactoryMetrics.get_metric(ensemble_corrpy)

Logger().reset()
for i in range(25):                      
    # training
    metrics = ensemble_corrpy.fit(input_train, target_train,
                                max_epoch=max_epoch, batch_size=32, early_stop=False)
    # Compute metrics
    metrics_ensemble_corrpy.append_prediction(input_test, target_test)
    metrics_ensemble_corrpy.append_prediction_per_model(input_test, target_test)
    metrics_ensemble_corrpy.append_metric(metrics)
    
    # Reset parameters of all ensemble's models
    ensemble_corrpy.reset()

print("FINISHED!")

### Results Ensemble - Correntropy

In [ ]:
metrics_ensemble_corrpy.classification_report()
metrics_ensemble_corrpy.diversity_report()
metrics_ensemble_corrpy.plot_confusion_matrix()
metrics_ensemble_corrpy.plot_cost(max_epoch, "Cost training Ensemble-Correntropy")
metrics_ensemble_corrpy.plot_costs(max_epoch, "Cost training Ensemble-Correntropy per models")
metrics_ensemble_corrpy.plot_scores(max_epoch, "Accuracy training Ensemble-Correntropy per models")

plt.tight_layout()